In [9]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

from cnn6 import CNN
from training import EvalLandmark
import CERDataset

from nni.retiarii.evaluator.pytorch import Lightning, Trainer
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

# mode settings for mode = 'pretrain2'
mode = 'pretrain2'
source_dir = 'TrainingImages2'
source_name = 'image_'
landmarks_dir = 'TrainingLabels1'
landmarks_name = 'labels_'
target_dir = 'DetectedLandmarks2'
target_name = 'imageDetected_'
params_name = '2pretrain_params.pt'
train_loss_name = '2pretrain_train_loss.npy'
val_loss_name = '2pretrain_val_loss.npy'
loss_fig = '2loss_Pretraining.png'
loss_file = '2loss.txt'

# landmarks
numOfLandmarks = 4

# settings for pytorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
torch.set_default_dtype(torch.float64)

model = CNN()
model.to(device)
model.load_state_dict(torch.load(params_name,map_location=torch.device(device)))
# model.eval()

# load and shuffle data and define train/test dataloader
full_data = CERDataset.CERDataset(landmark_dir=landmarks_dir, image_dir=source_dir)
train_data_size = int(0.72 * len(full_data))
test_data_size = int(0.18 * len(full_data))
valid_data_size = len(full_data) - train_data_size - test_data_size
assert train_data_size + test_data_size + valid_data_size == len(full_data)
train_data, test_data, valid_data = torch.utils.data.random_split(full_data,
                                                                  [train_data_size, test_data_size, valid_data_size],
                                                                  generator=torch.Generator().manual_seed(42))

train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=8, shuffle=False)

module = EvalLandmark(mode=mode)
early_stop_callback = EarlyStopping(
            monitor="validation_loss", 
            min_delta=0.00, 
            patience=200, 
            verbose=False, 
            mode="max"
            )
trainer = Trainer(
            max_epochs=10000,
            fast_dev_run=False,
            gpus=1,
            callbacks=[early_stop_callback],
            )
trainer.fit(model, train_dataloader, valid_dataloader)

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory